In [24]:
import keras
from keras import backend
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#!pip install tensorflow_hub
import tensorflow_hub as hub
import numpy as np
import shutil
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorboard
import os
import pandas as pd
import math

In [29]:
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5



In [30]:
batch_size = 16
img_height = 224
img_width = 224

In [32]:
dataset = 'Data/Processed_Data/train'
dataset_path = os.listdir(dataset)

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['train_ak_aug', 'train_bcc_aug', 'train_bkl_aug', 'train_df_aug', 'train_mel_aug', 'train_nv_aug', 'train_scc_aug', 'train_vasc_aug']
Types of classes labels found:  8


In [33]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# create a directory to save the files
create_dir("models")

In [34]:
# load the augmented data from the local data directory:

train_dir = "Data/Processed_Data/train"
test_dir = "Data/Processed_Data/test"
valid_dir = "Data/Processed_Data/valid_processed"

datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2, data_format="channels_last", dtype=tf.float32) # The imported data will br normalized here.
test_datagen = ImageDataGenerator(rescale = 1./255, data_format="channels_last", dtype=tf.float32)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,subset = 'training') # set as training data

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,subset = 'validation') # set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    shuffle = True,
    seed=123,) # set as testing data

Found 57680 images belonging to 8 classes.
Found 14416 images belonging to 8 classes.
Found 3799 images belonging to 8 classes.


In [35]:
# inception_resnet_v2
model = Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5",
               trainable=True, arguments=dict(batch_norm_momentum=0.997)),
    layers.Dense(8, activation='softmax'),
])

model.build([None, img_height, img_width, 3])  # Batch input shape.

In [36]:
METRICS = [
    keras.metrics.Accuracy(name= "accuracy"),
    keras.metrics.Precision(name = "precision"),
    keras.metrics.Recall(name = 'recall'),
    keras.metrics.AUC(name = 'auc'),
]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)


r = model.fit(
    train_generator,
    epochs = 2,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_data = validation_generator,
    validation_steps=validation_generator.samples//batch_size,
)

Epoch 1/2


NotFoundError: Graph execution error:

2 root error(s) found.
  (0) NOT_FOUND:  No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine 1#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 20318464 bytes.
  Profiling failure on CUDNN engine 1: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 20318464 bytes.
  Profiling failure on CUDNN engine 0#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 16777216 bytes.
  Profiling failure on CUDNN engine 0: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 16777216 bytes.
  Profiling failure on CUDNN engine 2#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 20463616 bytes.
  Profiling failure on CUDNN engine 2: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 20463616 bytes.
  Profiling failure on CUDNN engine 5#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 489160704 bytes.
  Profiling failure on CUDNN engine 5: RESOURCE_EXHAUSTED: Out of memor

In [ ]:
model.save("models/model_inception_resnetv2_10.h5")

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['ak', 'bcc', 'bkl', 'df', 'mel','nv', 'scc', 'vasc']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


In [ ]:
model.evaluate(validation_generator, verbose=2)

In [ ]:
# Plot the ROC curve

def plot_roc(labels, data):
    predictions = model.predict(data)
    fp, tp, _ = roc_curve(labels, predictions)
    plt.plot(100*fp, 100*tp)
    plt.xlabel("False Positives [%]")
    plt.xlabel("True Positives [%]")
    plt.show()

In [ ]:
test_labels = np.array([])
num_batches =0

for _, y in test_generator:
    test_labels = np.append(test_labels, y)
    num_batches +=1
    if num_batches == math.ceil(test_generator.samples / batch_size):
        break

plot_roc(test_labels, test_generator)
